In [1]:
from src import *
out_folder = "2019-2023-20241029"
all_years = ["2019", "2020", "2021", "2022", "2023", "2024"]

### Annotated images metadata

In [2]:
# Annoatated Images
annot_imgs = glob.glob(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\*\screened_with_fish\original_images\*\*.png")

all_headers = pd.read_pickle(os.path.join(out_folder, "all_headers.pickle"))
df_imgs_annot = create_image_df_save_csv(annot_imgs, out_folder, year="annotated") # annotated (69832, 2)
df_annot = create_unpacked_images_metatada_df(all_headers, df_imgs_annot, ext=".png", year = "annotated", old_filenames = True) #all (45542, 155)

annotated (69831, 3)
annotated (69831, 158)


In [ ]:
def make_list_by_collect(df_annot, year):
    base_dir = f'Z:\\__Organized_Directories_InProgress\\Fish_annotations_based_on_YOLO\\{year}\\screened_with_fish\\original_images'
    df_annot_yr = df_annot[df_annot.year.isin([year])]
    grouped_df = df_annot_yr.groupby('collect_id').agg({'image_path': list  }).reset_index()     # Creating a list of values in column2
    return grouped_df, base_dir
def get_src_dst_from_grouped_df(grouped_df, idx, base_dir):
    src_lst = grouped_df.image_path[idx]
    cid = grouped_df.collect_id[idx]
    new_dir = os.path.join(base_dir, cid)
    return src_lst, new_dir
def mv_lst(src_lst, new_dir):
    if not os.path.exists(new_dir): os.makedirs(new_dir)
    for i, itm in enumerate(src_lst):
        print("moving", i, "out of", len(src_lst), end="\r")
        # print(new_dir)
        if not os.path.exists(os.path.join(new_dir, os.path.basename(itm))):
            shutil.move(itm, new_dir)
        else: continue
def iterate_mv(df_annot, year):
    grouped_df, base_dir = make_list_by_collect(df_annot, year)
    for idx in range(len(grouped_df)):
        print("idx", idx, "out of", len(grouped_df), end="\r")
        src_lst, new_dir = get_src_dst_from_grouped_df(grouped_df, idx, base_dir)
        mv_lst(src_lst, new_dir)

year = "2023"
# iterate_mv(df_annot, year)

### Shadis Assmts

In [4]:
assmt_files_lst =  glob.glob(r"Z:\Database\02__MetadataCombined\*_assessment_confirmedfish.csv")
df_list = [pd.read_csv(file, low_memory=False) for file in assmt_files_lst]
df_assmt = pd.concat(df_list, ignore_index=True)
df_assmt = df_assmt.rename(columns={"Image_Name":"old_filename"})
df_assmt.columns.values

array(['old_filename', 'year', 'img_size', 'W', 'H', 'BLX', 'BLY', 'TLX',
       'TLY', 'TRX', 'TRY', 'BRX', 'BRY', 'Length_to_Width_Ratio',
       'MERR_Diagonal_Length', 'Bounding_Rect_Diagonal_Length', 'fish_id',
       'PS_mm', 'ImageArea_m2', 'date_', 'SF', 'Ell_MA_px', 'Ell_MA_mm',
       'Ell_Weight_g', 'Cir_D_px', 'Cir_D_mm', 'Cir_Weight_g',
       'Poly_Len_px', 'Poly_Len_mm', 'Poly_Weight_g', 'Poly_Corr_len_mm',
       'Poly_Corr_weight_g', 'CreationTime', 'AUV', 'Lake', 'Site',
       'cam_sys', 'DataSource', 'VehicleID', 'BagFile', 'MissionName',
       'Time_UTC', 'Usability', 'substrate_class', 'Latitude',
       'Longitude', 'Temperature_C', 'DepthFromSurface_m',
       'DistanceToBottom_m', 'TotalWaterColumn_m', 'CollectID'],
      dtype=object)

### check for missing metadata and output a list of images for each collect

In [ ]:
missing_metadata = df_annot[df_annot.Time_s.isna()].copy()
print("images with no metadata created", missing_metadata.shape)
# imgs_missing_metadata = missing_metadata["filename"]
# imgs_dfs = pd.concat([pd.read_csv(os.path.join(out_folder, "2022_imgs.csv")), pd.read_csv(os.path.join(out_folder, "2023_imgs.csv"))])
# collect_id_df = create_collect_id_df(imgs_dfs.image_path.values)
# collect_id_df.filename = collect_id_df.filename.apply(lambda x: x.replace("CI", "PI"))
# imgs_missing_metadata = pd.merge(imgs_missing_metadata, collect_id_df, on=["filename"], how="left")
# Group by 'column1' and create a list of all values in 'column2' for each group
# grouped_df = imgs_missing_metadata.groupby('collect_id').agg({'filename': list  }).reset_index()     # Creating a list of values in column2
# grouped_df.to_csv("images_missing_metadata.csv")
# grouped_df


images with no metadata created (0, 158)


### adding luke/andrew qaqc on dominant fish type

In [63]:
qaqc_files_lst = glob.glob(r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\datasets\alewife_qaqc\*_qaqc_complete.xlsx")
df_list = [pd.read_excel(file, index_col=0) for file in qaqc_files_lst]
alewife_qaqc = pd.concat(df_list, ignore_index=True)
alewife_qaqc["old_filename"] = alewife_qaqc.image_id.apply(lambda x: x+".png")
alewife_qaqc= alewife_qaqc[["old_filename", "fish_type"]]
'''fish_type
goby       1553
alewife      10
Name: count, dtype: int64'''
alewife_qaqc.fish_type.value_counts()

fish_type
goby       1553
alewife      10
Name: count, dtype: int64

In [64]:
alewife_qaqc

,old_filename,fish_type
0,image_raw_00431895.png,goby
1,image_raw_00435914.png,goby
2,image_raw_01391074.png,goby
3,image_raw_02227331.png,goby
4,image_raw_02230553.png,goby
...,...,...
1558,CI_1692556107_250.png,goby
1559,CI_1693084116_112.png,goby
1560,CI_1693156380_728.png,goby
1561,CI_1694284092_046.png,goby


### adding fish_type, and substrate tags

In [65]:
df_annot2 = pd.merge(df_annot, alewife_qaqc[["old_filename", "fish_type"]], on="old_filename", how ="left")
fish_type_na = df_annot2[df_annot2.fish_type.isna()].index
df_annot2.loc[fish_type_na,"fish_type"] = "not_checked"
substrate = pd.read_csv(r"Z:\Database\07__Metadata+Label\Annotated_imgs_substrate_predictions.csv", index_col=0)
substrate = substrate.rename(columns={'label': 'substrate_class'})
substrate['old_filename'] = substrate.image_path.apply(lambda x: os.path.basename(x))
df_annot3 = pd.merge(df_annot2, substrate[["old_filename", "substrate_class"]], on="old_filename", how ="left")
assert df_annot3[df_annot3.substrate_class.isna()].empty
assert df_annot3[df_annot3.fish_type.isna()].empty
df_annot3.to_csv("annotated_imgs_metadata_all_years_subs_fish_type.csv")
df_annot3.iloc[:,[0,1,2,3,4,5,-5,-4,-3,-2,-1]].head()

,Time_s,filename,Latitude,Longitude,DepthFromSurface_m,DistanceToBottom_m,time,image_path,old_filename,fish_type,substrate_class
0,1.596710e+09,PI_1596710233_220.png,44.978776,-85.517088,2.750677,1.869549,06:37:13,Z:\__Organized_Directories_InProgress\Fish_ann...,image_raw_10233220.png,not_checked,coarse
1,1.596710e+09,PI_1596710234_024.png,44.978781,-85.517078,2.786023,1.843977,06:37:14,Z:\__Organized_Directories_InProgress\Fish_ann...,image_raw_10234024.png,not_checked,coarse
2,1.596710e+09,PI_1596710235_632.png,44.978791,-85.517060,2.800000,1.840000,06:37:15,Z:\__Organized_Directories_InProgress\Fish_ann...,image_raw_10235632.png,not_checked,coarse
3,1.596710e+09,PI_1596710236_439.png,44.978795,-85.517053,2.788075,1.870000,06:37:16,Z:\__Organized_Directories_InProgress\Fish_ann...,image_raw_10236439.png,not_checked,coarse
4,1.596710e+09,PI_1596710237_239.png,44.978800,-85.517044,2.766336,1.890496,06:37:17,Z:\__Organized_Directories_InProgress\Fish_ann...,image_raw_10237239.png,not_checked,coarse


In [66]:
df_annot3.fish_type.value_counts()

fish_type
not_checked    68304
goby            1553
alewife           10
Name: count, dtype: int64

In [67]:
in_my_not_shadi = set(df_annot.old_filename).difference(df_assmt.old_filename)
in_shadi_not_my = set(df_assmt.old_filename).difference(df_annot.old_filename)
len(in_shadi_not_my)

1

In [68]:
test = pd.merge(df_annot3[["old_filename", "DistanceToBottom_m"]], df_assmt[["DistanceToBottom_m", "old_filename"]], on = "old_filename")
test["diffr"] = test.DistanceToBottom_m_x - test.DistanceToBottom_m_y
test[(test.diffr < -0.1) |  (test.diffr > 0.1)]

,old_filename,DistanceToBottom_m_x,DistanceToBottom_m_y,diffr
50212,image_raw_70514481.png,2.045984,1.833452,0.212532
50213,image_raw_70514481.png,2.045984,1.833452,0.212532
54899,image_raw_75737346.png,2.203771,1.438117,0.765654
54900,image_raw_75737346.png,2.203771,1.438117,0.765654
54901,image_raw_75737346.png,2.203771,1.438117,0.765654
...,...,...,...,...
231504,CI_1694456683_603.png,2.580000,2.905411,-0.325411
231505,CI_1694457010_603.png,2.695992,2.900375,-0.204383
231506,CI_1694457136_103.png,2.690000,2.898442,-0.208442
231507,CI_1694458329_103.png,2.580000,2.880069,-0.300069


In [ ]:
shadis_columns = ['old_filename', 'img_size', 'W', 'H', 'BLX', 'BLY', 'TLX',
                'TLY', 'TRX', 'TRY', 'BRX', 'BRY', 'Length_to_Width_Ratio',
                'MERR_Diagonal_Length', 'Bounding_Rect_Diagonal_Length', 'fish_id',
                'PS_mm', 'ImageArea_m2', 'date_', 'SF', 'Ell_MA_px', 'Ell_MA_mm',
                'Ell_Weight_g', 'Cir_D_px', 'Cir_D_mm', 'Cir_Weight_g',
                'Poly_Len_px', 'Poly_Len_mm', 'Poly_Weight_g', 'Poly_Corr_len_mm',
                'Poly_Corr_weight_g', 'CreationTime', 'AUV', 'Lake', 'Site',
                'cam_sys']
                
my_columns = ['Time_s', 'old_filename', 'filename', 'Latitude', 'Longitude',
                'DepthFromSurface_m', 'DistanceToBottom_m', 'TotalWaterColumn_m',
                'Speed_kn', 'Time_UTC', 'DataSource', 'HeadingCompass_deg',
                'PitchCompass_deg', 'RollCompass_deg', 'TempCompass_C',
                'DepthCompass_bar', 'year', 'month', 'day', 'time', 'image_path',
                'old_filename', 'fish_type', 'substrate_class', 'Usability', 'collect_id']

output_columns = ['Time_s', 'filename', 'old_filename', 'year', 'img_size', 'W', 'H', 'BLX', 'BLY', 'TLX',
                    'TLY', 'TRX', 'TRY', 'BRX', 'BRY', 'Length_to_Width_Ratio',
                    'MERR_Diagonal_Length', 'Bounding_Rect_Diagonal_Length', 'fish_id',
                    'PS_mm', 'ImageArea_m2', 'date_', 'SF', 'Ell_MA_px', 'Ell_MA_mm',
                    'Ell_Weight_g', 'Cir_D_px', 'Cir_D_mm', 'Cir_Weight_g',
                    'Poly_Len_px', 'Poly_Len_mm', 'Poly_Weight_g', 'Poly_Corr_len_mm',
                    'Poly_Corr_weight_g', 'CreationTime', 'AUV', 'Lake', 'Site',
                    'cam_sys', 'DataSource', 'VehicleID', 'BagFile', 'MissionName',
                    'Time_UTC', 'Usability', 'substrate_class', 'Latitude',
                    'Longitude', 'Temperature_C', 'DepthFromSurface_m',
                    'DistanceToBottom_m', 'TotalWaterColumn_m', 'collect_id']